# YOLOv5 on Custom Objects

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference

In [1]:
!pip install wandb -qqq
import wandb
# wandb.login()
# wandb.init(project="YOLOv5")

In [1]:
import wandb
wandb.init(project="YOLOv5_Rosemary")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: angelajlee. Use `wandb login --relogin` to force relogin


In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/lab03/EvolveYOLO/yolov5
HEAD is now at 886f1c0 DDP after autoanchor reorder (#2421)


In [3]:
!cd ./
!pwd

/home/lab03/EvolveYOLO/yolov5


In [4]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.
Setup complete. Using torch 1.8.1+cu111 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [5]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")

You should consider upgrading via the '/home/ubuntu/anaconda3/envs/pytorch_latest_p37/bin/python -m pip install --upgrade pip' command.
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=roboflow-yolov5


In [30]:
%cd ./
#after following the link above, recieve python code with these fields filled in

# Version 1 metrics/mAP_0.5 0.04278
# from roboflow import Roboflow
# rf = Roboflow(api_key="QWencAPU4nMRzIBHlDhp")
# project = rf.workspace("smart-pot").project("test-sample-npc1h")
# dataset = project.version(2).download("yolov5")

# Version 2 metrics/mAP_0.5 0.34993
from roboflow import Roboflow
rf = Roboflow(api_key="IBBTtAIXCLhJqbn75Abw")
project = rf.workspace("mulcam").project("rosemary-umdoq")
dataset = project.version(5).download("yolov5")

# Version 3 metrics/mAP_0.5 0.23555
# from roboflow import Roboflow
# rf = Roboflow(api_key="QWencAPU4nMRzIBHlDhp")
# project = rf.workspace("smart-pot").project("rosemary-yqkfo")
# dataset = project.version(5).download("yolov5")

/home/lab03/EvolveYOLO/yolov5
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Rosemary-5 in yolov5pytorch:: 100%|██████████| 2540/2540 [00:00<00:00, 4874.97it/s]


In [7]:
# curl -L "https://app.roboflow.com/ds/EbfTvnORki?key=IHIDfi7nN4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [33]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names: ['Rosemary', 'Rosemary Leaf Spot', 'Rosemary Pest Damage', 'Rosemary Powdery Mildew']
nc: 4
train: Rosemary-5/train/images
val: Rosemary-5/valid/images
test: Rosemary-5/test/images

## Count images on the Dataset

In [34]:
!find {dataset.location} -name *.jpg | wc -l
!find {dataset.location}/train -name *.jpg | wc -l
!find {dataset.location}/valid -name *.jpg | wc -l
!find {dataset.location}/test -name *.jpg | wc -l

2533
1996
358
179


# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [11]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [12]:
#this is the model configuration we will use for our tutorial 
%cat ./models/yolov5s.yaml
# %cat ./models/yolov5m.yaml
# %cat ./models/yolov5x.yaml
# %cat ./models/hub/yolov5l6.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C

In [13]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [14]:
%%writetemplate ./models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [15]:
# %%writetemplate ./models/custom_yolov5l6.yaml

# # parameters
# nc: {num_classes}  # number of classes
# depth_multiple: 1.0  # model depth multiple
# width_multiple: 1.0  # layer channel multiple

# # anchors
# anchors:
#   - [ 19,27,  44,40,  38,94 ]  # P3/8
#   - [ 96,68,  86,152,  180,137 ]  # P4/16
#   - [ 140,301,  303,264,  238,542 ]  # P5/32
#   - [ 436,615,  739,380,  925,792 ]  # P6/64

# # YOLOv5 backbone
# backbone:
#   # [from, number, module, args]
#   [ [ -1, 1, Focus, [ 64, 3 ] ],  # 0-P1/2
#     [ -1, 1, Conv, [ 128, 3, 2 ] ],  # 1-P2/4
#     [ -1, 3, C3, [ 128 ] ],
#     [ -1, 1, Conv, [ 256, 3, 2 ] ],  # 3-P3/8
#     [ -1, 9, C3, [ 256 ] ],
#     [ -1, 1, Conv, [ 512, 3, 2 ] ],  # 5-P4/16
#     [ -1, 9, C3, [ 512 ] ],
#     [ -1, 1, Conv, [ 768, 3, 2 ] ],  # 7-P5/32
#     [ -1, 3, C3, [ 768 ] ],
#     [ -1, 1, Conv, [ 1024, 3, 2 ] ],  # 9-P6/64
#     [ -1, 1, SPP, [ 1024, [ 3, 5, 7 ] ] ],
#     [ -1, 3, C3, [ 1024, False ] ],  # 11
#   ]

# # YOLOv5 head
# head:
#   [ [ -1, 1, Conv, [ 768, 1, 1 ] ],
#     [ -1, 1, nn.Upsample, [ None, 2, 'nearest' ] ],
#     [ [ -1, 8 ], 1, Concat, [ 1 ] ],  # cat backbone P5
#     [ -1, 3, C3, [ 768, False ] ],  # 15

#     [ -1, 1, Conv, [ 512, 1, 1 ] ],
#     [ -1, 1, nn.Upsample, [ None, 2, 'nearest' ] ],
#     [ [ -1, 6 ], 1, Concat, [ 1 ] ],  # cat backbone P4
#     [ -1, 3, C3, [ 512, False ] ],  # 19

#     [ -1, 1, Conv, [ 256, 1, 1 ] ],
#     [ -1, 1, nn.Upsample, [ None, 2, 'nearest' ] ],
#     [ [ -1, 4 ], 1, Concat, [ 1 ] ],  # cat backbone P3
#     [ -1, 3, C3, [ 256, False ] ],  # 23 (P3/8-small)

#     [ -1, 1, Conv, [ 256, 3, 2 ] ],
#     [ [ -1, 20 ], 1, Concat, [ 1 ] ],  # cat head P4
#     [ -1, 3, C3, [ 512, False ] ],  # 26 (P4/16-medium)

#     [ -1, 1, Conv, [ 512, 3, 2 ] ],
#     [ [ -1, 16 ], 1, Concat, [ 1 ] ],  # cat head P5
#     [ -1, 3, C3, [ 768, False ] ],  # 29 (P5/32-large)

#     [ -1, 1, Conv, [ 768, 3, 2 ] ],
#     [ [ -1, 12 ], 1, Concat, [ 1 ] ],  # cat head P6
#     [ -1, 3, C3, [ 1024, False ] ],  # 32 (P6/64-xlarge)

#     [ [ 23, 26, 29, 32 ], 1, Detect, [ nc, anchors ] ],  # Detect(P3, P4, P5, P6)
#   ]

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [28]:
%cat ./utils/google_utils.py

# Google utils: https://cloud.google.com/storage/docs/reference/libraries

import os
import platform
import subprocess
import time
from pathlib import Path

import requests
import torch


def gsutil_getsize(url=''):
    # gs://bucket/file size https://cloud.google.com/storage/docs/gsutil/commands/du
    s = subprocess.check_output(f'gsutil du {url}', shell=True).decode('utf-8')
    return eval(s.split(' ')[0]) if len(s) else 0  # bytes


def attempt_download(file, repo='ultralytics/yolov5'):
    # Attempt file download if does not exist
    file = Path(str(file).strip().replace("'", '').lower())

    if not file.exists():
        try:
            response = requests.get(f'https://api.github.com/repos/{repo}/releases/tags/v5.0').json() # github api
            assets = [x['name'] for x in response['assets']]  # release assets, i.e. ['yolov5s.pt', 'yolov5m.pt', ...]
            tag = response['tag_name']  # i.e. 'v1.0'
        except:  # fallback plan
            assets = ['yolov5s.pt', 

In [17]:
%cat ./data/hyp.scratch.yaml

# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  #

In [18]:
# %cd ./
# !python train.py -h

In [19]:
# wandb.init(project="YOLOv5",
#            config={
#                "batch_size": 32,
#                "learning_rate": 0.01,
#                "dataset": "Rosemary-5",
#            })

## Version 1 metrics/mAP_0.5 0.04278
## Version 2 metrics/mAP_0.5 0.34993
## Version 3 metrics/mAP_0.5 0.23555

In [35]:
%%time

%%wandb

%cd ./
!python train.py \
--img 416 \
--batch 8 \
--epochs 10 \
--data {dataset.location}/data.yaml \
--cfg ./models/custom_yolov5s.yaml \
--weights yolov5s.pt \
--name yolov5s_rosemary \
--cache \
--hyp "./data/hyp.scratch.yaml" \
# --resume \

/home/lab03/EvolveYOLO/yolov5
github: ⚠️ WARNING: code is out of date by 1118 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.8.1+cu111 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='/home/lab03/EvolveYOLO/yolov5/Rosemary-5/data.yaml', device='', entity=None, epochs=10, evolve=False, exist_ok=False, global_rank=-1, hyp='./data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='yolov5s_rosemary', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5s_rosemary', single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov5s.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at ht

# Hyperparameter Evolution (Optional. 300 scenarios)

In [37]:
%%time

!python train.py \
--img 416 \
--batch 8 \
--epochs 10 \
--data {dataset.location}/data.yaml \
--cfg ./models/custom_yolov5s.yaml \
--weights yolov5s.pt \
--project YOLOv5_Rosemary \
--name rosemary_evolve \
--cache \
--hyp "./data/hyp.scratch.yaml" \
--evolve

github: ⚠️ WARNING: code is out of date by 1118 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-126-g886f1c0 torch 1.8.1+cu111 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=True, cfg='./models/custom_yolov5s.yaml', data='/home/lab03/EvolveYOLO/yolov5/Rosemary-5/data.yaml', device='', entity=None, epochs=10, evolve=True, exist_ok=False, global_rank=-1, hyp='./data/hyp.scratch.yaml', image_weights=False, img_size=[416, 416], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='evolve', noautoanchor=False, nosave=False, notest=False, project='runs', quad=False, rect=False, resume=False, save_dir='runs/evolve', single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov5s.pt', workers=8, world_size=1)
hyperparameters: lr0=0.0152, lrf=0.2565, momentum=0.765, weight_decay=0.00021, warmup_epochs=1.63, warmup_momentum=0.61014, warmup_

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      metrics/mAP_0.5 ▁▁▁▁▁▁▁▁▁█
wandb: metrics/mAP_0.5:0.95 ▁▁▁▁▁▁▁▁▁█
wandb:    metrics/precision ▁▁▁▁▁▁▁▁▁█
wandb:       metrics/recall ▁▁▁▁▁▁▁▁▁█
wandb:       train/box_loss █▇▄▃▂▂▂▁▁▁
wandb:       train/cls_loss █▆▃▃▃▃▃▂▂▁
wandb:       train/obj_loss ▃▄▆██▇▆▄▃▁
wandb:         val/box_loss ▁▁▁▁▁▁▁▁▁█
wandb:         val/cls_loss ▁▁▁▁▁▁▁▁▁█
wandb:         val/obj_loss ▁▁▁▁▁▁▁▁▁█
wandb:                x/lr0 ▁▄▆█▇▇▅▄▃▂
wandb:                x/lr1 ▁▄▆█▇▇▅▄▃▂
wandb:                x/lr2 █▆▄▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:      metrics/mAP_0.5 0.18556
wandb: metrics/mAP_0.5:0.95 0.12977
wandb:    metrics/precision 0.21558
wandb:       metrics/recall 0.3801
wandb:       train/box_loss 0.01205
wandb:       train/cls_loss 0.01963
wandb:       train/obj_loss 0.01331
wandb:         val/box_loss 0.01688
wandb:  

       1/9    0.805G   0.02928   0.02729   0.03031   0.08689         7       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       2/9    0.805G   0.02075   0.03042   0.02918   0.08036        11       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       3/9    0.805G   0.01996   0.02819   0.02893   0.07708        11       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       4/9    0.805G   0.01897    0.0259   0.02882   0.07369         5       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       5/9    0.805G   0.01802   0.02331   0.02855   0.06988         8       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       6/9    0.805G   0.01649   0.02064   0.02843   0.06556         2       416

     Epoch   gpu_mem       box       obj       cls     total   targets  img_size
       7/9    0.805G  

train: Scanning 'Rosemary-5/train/labels.cache' for images and labels... 1996 fo
train: Caching images (1.0GB): 100%|███████| 1996/1996 [00:02<00:00, 823.60it/s]
val: Scanning 'Rosemary-5/valid/labels.cache' for images and labels... 358 found

autoanchor: Analyzing anchors... anchors/target = 0.00, Best Possible Recall (BPR) = 0.0000. Attempting to improve anchors, please wait...
autoanchor: Running kmeans for 9 anchors on 2398 points...
autoanchor: thr=0.37: 0.9996 best possible recall, 8.06 anchors past thr
autoanchor: n=9, img_size=416, metric_all=0.589/0.868-mean/best, past_thr=0.622-mean: 111,142,  189,162,  157,244,  291,212,  176,356,  235,295,  357,285,  298,362,  384,390
autoanchor: Evolving anchors with Genetic Algorithm: fitness = 0.8715: 100%|█| 1
autoanchor: thr=0.37: 1.0000 best possible recall, 8.01 anchors past thr
autoanchor: n=9, img_size=416, metric_all=0.588/0.871-mean/best, past_thr=0.623-mean: 103,140,  168,161,  147,239,  263,212,  172,342,  232,302,  338,273,  2

 23                -1  1   1248768  models.common.BottleneckCSP             [512, 512, 1, False]          
 24      [17, 20, 23]  1     24273  models.yolo.Detect                      [4, [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]], [128, 256, 512]]
Model Summary: 283 layers, 7263185 parameters, 7263185 gradients, 16.9 GFLOPS

Transferred 258/370 items from yolov5s.pt
Scaled weight_decay = 0.0002
Optimizer groups: 62 .bias, 70 conv.weight, 59 other
wandb: Tracking run with wandb version 0.12.18
wandb: Run data is saved locally in /home/lab03/EvolveYOLO/yolov5/wandb/run-20220621_235001-2wubhp8p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run evolve
wandb: ⭐️ View project at https://wandb.ai/angelajlee/runs
wandb: 🚀 View run at https://wandb.ai/angelajlee/runs/runs/2wubhp8p
train: Scanning 'Rosemary-5/train/labels.cache' for images and labels... 1996 fo
train: Caching images (1.0GB): 100%|███████| 1996/1996 [00:02<00:00, 801.36it/s]
val: Scanning 'Rosem

...

  6                -1  1    641792  models.common.BottleneckCSP             [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1    656896  models.common.SPP                       [512, 512, [5, 9, 13]]        
  9                -1  1   1248768  models.common.BottleneckCSP             [512, 512, 1, False]          
 10                -1  1    131584  models.common.Conv                      [512, 256, 1, 1]              
 11                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
 12           [-1, 6]  1         0  models.common.Concat                    [1]                           
 13                -1  1    378624  models.common.BottleneckCSP             [512, 256, 1, False]          
 14                -1  1     33024  models.common.Conv                      [256, 128, 1, 1]              
 15                -1  1         0  t

hyperparameters: lr0=0.0152, lrf=0.19521, momentum=0.7722, weight_decay=0.00019, warmup_epochs=1.77614, warmup_momentum=0.79358, warmup_bias_lr=0.13479, box=0.0246, cls=0.41305, cls_pw=0.74263, obj=1.03148, obj_pw=0.79516, iou_t=0.2, anchor_t=2.44095, anchors=2.96752, fl_gamma=0.0, hsv_h=0.01643, hsv_s=0.72341, hsv_v=0.31391, degrees=0.0, translate=0.0659, scale=0.36794, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=0.62943, mixup=0.0
Overriding model.yaml anchors with anchors=2.96752

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv              

hyperparameters: lr0=0.01468, lrf=0.207, momentum=0.75626, weight_decay=0.00024, warmup_epochs=1.63, warmup_momentum=0.83061, warmup_bias_lr=0.14376, box=0.0244, cls=0.42887, cls_pw=0.853, obj=0.96808, obj_pw=0.74293, iou_t=0.2, anchor_t=2.54366, anchors=2.88, fl_gamma=0.0, hsv_h=0.01495, hsv_s=0.76559, hsv_v=0.35882, degrees=0.0, translate=0.06268, scale=0.3744, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=0.53564, mixup=0.0
Overriding model.yaml anchors with anchors=2.88

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64

hyperparameters: lr0=0.0152, lrf=0.19333, momentum=0.76393, weight_decay=0.00021, warmup_epochs=1.78799, warmup_momentum=0.75812, warmup_bias_lr=0.12491, box=0.02596, cls=0.38818, cls_pw=0.853, obj=0.94427, obj_pw=0.74047, iou_t=0.2, anchor_t=2.50906, anchors=2.07868, fl_gamma=0.0, hsv_h=0.0164, hsv_s=0.71924, hsv_v=0.347, degrees=0.0, translate=0.07189, scale=0.35563, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=0.5427, mixup=0.0
Overriding model.yaml anchors with anchors=2.07868

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     19904  models.common.BottleneckCSP             [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                 

wandb:         val/obj_loss ▁▁▁▁▁▁▁▁▁█
wandb:                x/lr0 ▁▄▆█▇▇▅▄▂▁
wandb:                x/lr1 ▁▄▆█▇▇▅▄▂▁
wandb:                x/lr2 █▆▄▂▂▂▁▁▁▁
wandb: 
wandb: Run summary:
wandb:      metrics/mAP_0.5 0.34099
wandb: metrics/mAP_0.5:0.95 0.22456
wandb:    metrics/precision 0.29503
wandb:       metrics/recall 0.47168
wandb:       train/box_loss 0.01292
wandb:       train/cls_loss 0.022
wandb:       train/obj_loss 0.01514
wandb:         val/box_loss 0.01956
wandb:         val/cls_loss 0.02083
wandb:         val/obj_loss 0.01194
wandb:                x/lr0 0.00411
wandb:                x/lr1 0.00411
wandb:                x/lr2 0.00411
wandb: 
wandb: Synced evolve: https://wandb.ai/angelajlee/runs/runs/2b6b7qnw
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20220622_005916-2b6b7qnw/logs

       lr0       lrf  momentumweight_decaywarmup_epochswarmup_momentumwarmup_bias_lr       box       cls    cls_pw       obj

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# !kill 17409

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
# %load_ext tensorboard
# # %reload_ext tensorboard
# %tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='./runs/train/yolov5l6_rosemary4/results.png', width=1000)  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# print out an augmented training example / Mosaic Augmentation 적용된 train 이미지
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='./runs/train/yolov5l6_rosemary/train_batch0.jpg', width=900)

In [ ]:
# display our ground truth data / test 데이터 정답값
print("GROUND TRUTH TEST DATA:")
Image(filename='./runs/train/yolov5l6_rosemary/test_batch0_labels.jpg', width=900)

In [ ]:
# display our prediction / test 데이터 예측 결과값
print("PREDICT TEST DATA:")
Image(filename='./runs/train/yolov5l6_rosemary/test_batch0_pred.jpg', width=900)

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls ./runs

In [ ]:
%ls ./runs/train/yolov5l6_rosemary/weights

In [ ]:
!python ./detect.py -h

In [ ]:
from IPython.display import Image

!cd ./
!python detect.py --weights runs/train/yolov5l6_rosemary/weights/best.pt --img 256 --conf 0.4 --source data/images/rosemarytest1.jpg
Image(filename='./runs/detect/exp/rosemarytest1.jpg', width=600)

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd ./
!python detect.py --weights runs/train/yolov5l6_rosemary/weights/best.pt --img 256 --conf 0.4 --source Rosemary-3/test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('./runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Computing the mAP on the test dataset

In [ ]:
%cd ./
!python test.py -h

In [ ]:
%cd ./
!python test.py --weights runs/train/yolov5l6_rosemary/weights/best.pt --data models/custom_yolov5s.yaml --img 256 --name yolo_test_rosemary --task 'test'

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
%cd ./
# %mkdir YOLO v5 Weights
%cp ./runs/train/yolov5l6_rosemary/weights/best.pt /home/lab03/MultiCampus/YOLOv5Weights/

## Congrats!

Hope you enjoyed this!

--Team [Roboflow](https://roboflow.ai)